In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from sklearn.cluster import DBSCAN

In [2]:
_bb=pd.read_csv('bkk_budget_ocr.csv',index_col=0)
_bb=_bb[~_bb['pagenum'].isna()]

/Users/napatswift/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# number of document
len(_bb.pdf.unique())

385

In [4]:
_bb['pagenum'] = _bb['pagenum'].astype(int)
_bb['text'] = _bb['text'].astype(str)

select test set

In [5]:
interested_doc = [
    'pdf/61/A20171003163303.pdf', 
    'pdf/61/A20171003164329.pdf', 
    'pdf/61/A20171003170601.pdf', 
    'pdf/62/A20181002170334.pdf',
    'pdf/64/topic2787_2020_10_06_10_18_31.pdf',
    'pdf/64/topic2787_2020_10_06_10_18_51.pdf']

create total sum of the document for validation

In [6]:
answers = {
    'pdf/61/A20171003163303.pdf': {'sum':691_229_200, 'start': 3},
    'pdf/61/A20171003164329.pdf': {'sum': 386_943_640, 'start': 5},
    'pdf/61/A20171003170601.pdf': {'sum': 365_299_840, 'start': 5},
    'pdf/62/A20181002170334.pdf': {'sum': 357_892_000, 'start': 5},
    'pdf/64/topic2787_2020_10_06_10_18_31.pdf': {'sum': 424_674_000, 'start': 5},
    'pdf/64/topic2787_2020_10_06_10_18_51.pdf': {'sum': 398_016_000, 'start': 5},
    }

remove unwanted document (just for develop)

In [7]:
bb=_bb[_bb.pdf.isin(interested_doc)]

fix text by dict

In [8]:
chr_fix_pair={'ำ': 'า','/า': 'ำ', '/้า': '้ำ'}

In [9]:
def fix_pdf_text(old_text):
    new_text = old_text.strip()
    for o,n in chr_fix_pair.items():
        new_text=new_text.replace(o,n)
    return new_text

bb=bb.assign(fix_text=bb.text.apply(fix_pdf_text))

replace text from pdf with OCR text

In [10]:
bb.loc[~bb['ocr-text'].isna(), 'fix_text'] = bb[~bb['ocr-text'].isna()]['ocr-text']

group line

In [11]:
max_dist = 0.7
dbscan = DBSCAN(eps=max_dist, min_samples=0, metric='precomputed')
for g, idx in tqdm(bb.groupby('image_path').groups.items()):
    out = bb.loc[idx][['x0', 'x1', 'y0', 'y1']]
    out['h'] = out.y1 - out.y0
    num_bboxes = len(idx)
    idx2idx = {j:i for i,j in enumerate(idx)}
    n_dist_mat = np.ones((num_bboxes, num_bboxes))
    dist_mat = np.ones((num_bboxes, num_bboxes))
    
    for i in range(num_bboxes):
        a = out.iloc[i]
        _out=out.loc[idx[i:]]
        _skip = (a.y0 > _out.y1) | (_out.y0 > a.y1)
        _out = _out[~_skip]
        intersection = np.minimum(a.y1, _out.y1) - np.maximum(a.y0, _out.y0)
        union = a.h + _out.h - intersection
        iou = np.clip(intersection / (union + 1e-10), 0, 1)
        idicies = [idx2idx[i] for i in iou.index]
        n_dist_mat[i, idicies] = n_dist_mat[idicies, i] = 1-iou.values

    fdbscan = dbscan.fit(n_dist_mat)
    bb.loc[idx, 'line_label'] = fdbscan.labels_

100%|█████████████████████████████████████████| 221/221 [00:54<00:00,  4.02it/s]


In [12]:
bb.loc[:, 'line_label'] = bb.line_label.astype(int)

# Try to parse

In [46]:
import re
def get_patern_of_bullet(String):
    regx = [
        ('^0[-\d]+$', 70),
        ('^\(\d*(\.?\d*)*\)$', 50),
        ('^[1-9]\d*(\.[1-9]\d*)*\)$', 20),
        ('^[1-9]\d*(\.[1-9]\d*)+$', 5),
        ('^[1-9]\d*\.$', 4),
        ('^งาน', 3),
        ('^แผนงาน', 2),
        ('^[\u0E00-\u0E7F]\.', 1)
    ]
    
    if String in ['ด้านการจัดบริการของสำนักงานเขต',
                  'ด้านการบริหารจัดการและบริหารราชการกรุงเทพมหานคร',
                  'ด้านการศึกษา',
                  'ด้านความปลอดภัยและความเป็นระเบียบเรียบร้อย',
                  'ด้านทรัพยากรธรรมชาติและสิ่งแวดล้อม',
                  'ด้านพัฒนาสังคมและชุมชนเมือง',
                  'ด้านสาธารณสุข',
                  'ด้านเมืองและการพัฒนาเมือง',
                  'ด้านเศรษฐกิจและการพาณิชย์']: return ('ด้าน',1)

    for r, l in regx:
        if re.match(r, String):
            if l in [5, 20, 50]:
                l = String.count('.') + l
            if r == '^งาน' and String == 'งานที่จะทำ': continue
            return r, l
    return '', 0

In [47]:
bb_with_entry = bb.copy()

In [48]:
entry_label_id = 0

is_entry = False
for (pdf_name, pdf_page_index), page_df in tqdm(bb[~bb.line_label.isna()].groupby(['pdf','pagenum'])):
    if pdf_page_index + 1 < answers[pdf_name]['start']: continue
    for line_num, line_df in page_df.groupby('line_label'):
        if (line_df.fix_text == 'รายละเอียดรายจ่าย').any(): continue

        if (line_df.fix_text.isin(['ปี', '2560', 'บาท'])).sum() == 3:
            continue

        if line_num == 0: continue

        text_line = line_df.sort_values(by='x0',ascending=True).fix_text.values
        bullet_code = get_patern_of_bullet(text_line[0])[1]
        
        if bullet_code == 70:
            if len(text_line) > 1:
                second_bullet = get_patern_of_bullet(text_line[1])[1]
                if second_bullet != 0: bullet_code = second_bullet
        
        if bullet_code != 0 and line_df.x0.min() < 345:
            entry_label_id += 1
            bb_with_entry.loc[line_df.index, 'bullet_label'] = bullet_code
            is_entry = True

        if is_entry:
            bb_with_entry.loc[line_df.index, 'entry_label'] = entry_label_id

        is_last_token_baht = text_line[-1] == 'บาท'
        if is_last_token_baht:
            is_entry = False

100%|█████████████████████████████████████████| 221/221 [00:05<00:00, 36.99it/s]


In [49]:
# inspect them
start_entry=bb_with_entry[~bb_with_entry.bullet_label.isna()].groupby(['pdf','pagenum','entry_label']).x0.min()
start_entry.mean(),start_entry.std(),start_entry.mean()+start_entry.std(),start_entry.max()

(209.97351808547972, 42.14135666620478, 252.1148747516845, 272.2320098876953)

In [50]:
# bb_with_entry[~bb_with_entry.bullet_label.isna() & (bb_with_entry.x0 < 400 )].fix_text.values.tolist()

In [51]:
# test print by entry
bb_with_entry.iloc[:600]\
    .groupby(['pdf','pagenum', 'entry_label'])\
    .apply(lambda x: (x.iloc[0].entry_label,
                      np.unique(x.bullet_label.values.tolist()),
                      ' '.join(x.fix_text.values.tolist())
                     )
          )\
    .to_list();

Relation extraction

pairing parent node with child nodes

In [134]:
entry_bullet_table = bb_with_entry.groupby('entry_label').apply(lambda x: x.bullet_label.unique()[0])
entry_text_table = bb_with_entry.groupby('entry_label').apply(lambda x: ' '.join(x.fix_text.values))

In [152]:
child_parent_table = dict()
curr = None
for entry, bullet in entry_bullet_table.items():
    while (curr != None and bullet <= entry_bullet_table[curr]):
        curr = child_parent_table[curr]
    child_parent_table[entry] = curr
    curr = entry

for c, p in child_parent_table.items():
    print(None if p is None else entry_text_table.loc[p], entry_text_table.loc[c])
    if c > 30: break

None 01 ก. ด้านการบริหารทั่วไป 653,189,400 บาท
01 ก. ด้านการบริหารทั่วไป 653,189,400 บาท 0101 1. แผนงานบริหารทั่วไป 18,829,800 บาท
01 ก. ด้านการบริหารทั่วไป 653,189,400 บาท 0101001 งานบริหารทั่วไป ปี 2561 18,829,800 บาท
0101001 งานบริหารทั่วไป ปี 2561 18,829,800 บาท 01 1. เงินเดือนและค่าจ้างประจำ 14,612,000 บาท
01 1. เงินเดือนและค่าจ้างประจำ 14,612,000 บาท 01101-1 อัตราเดิม 29 อัตรา 11,230,400 บาท
01 1. เงินเดือนและค่าจ้างประจำ 14,612,000 บาท 01102-1 เงินเลื่อนขั้น 499,800 บาท
01 1. เงินเดือนและค่าจ้างประจำ 14,612,000 บาท 01106-1 เงินประจำตำแหน่ง 601,200 บาท
01 1. เงินเดือนและค่าจ้างประจำ 14,612,000 บาท 01107-1 เงินค่าตอบแทนรายเดือนของข้าราชการ 685,200 บาท
01 1. เงินเดือนและค่าจ้างประจำ 14,612,000 บาท 01201-1 อัตราเดิม 8 อัตรา 1,543,900 บาท
01 1. เงินเดือนและค่าจ้างประจำ 14,612,000 บาท 01202-1 เงินเพิ่มค่าจ้างประจำ 51,000 บาท
01 1. เงินเดือนและค่าจ้างประจำ 14,612,000 บาท 01205-1 เงินเพิ่มการครองชีพชั่วคราวของลูกจ้างประจำ 500 บาท
0101001 งานบริหารทั่วไป ปี 2561 18,829,800 บาท 03 2. ค่าต

In [150]:
for entry_label, entry_df in bb_with_entry.groupby('entry_label'):
    print(entry_df.fix_text.values)

['01' 'ก.' 'ด้านการบริหารทั่วไป' '653,189,400' 'บาท']


In [109]:
parent_child_key = dict()

for pdf_name, pdf_df in bb_with_entry.groupby(['pdf']):
    print(pdf_name)
    prev_bullet_label = None
    prev_entry_label = None
    
    entry_label_is_na = pdf_df.entry_label.isna()
    
    for entry, entry_df in pdf_df[~entry_label_is_na].groupby(['entry_label']):
        bullet_label = entry_df[~entry_df.bullet_label.isna()].bullet_label.unique()[0]
        entry_label = entry_df.entry_label.unique()[0]
        
        if prev_bullet_label is not None and bullet_label - prev_bullet_label > 0:
            parent_child_key[entry_label] = prev_entry_label

        prev_bullet_label = bullet_label
        prev_entry_label = entry_label

pdf/61/A20171003163303.pdf
pdf/61/A20171003164329.pdf
pdf/61/A20171003170601.pdf
pdf/62/A20181002170334.pdf
pdf/64/topic2787_2020_10_06_10_18_31.pdf
pdf/64/topic2787_2020_10_06_10_18_51.pdf


In [110]:
len(parent_child_key)

416